In [ ]:
import os
os.chdir("..")

In [ ]:
import polars as pl
from src.data.data_process import DataClean

dc = DataClean()

In [ ]:
df = dc.dataset()

In [ ]:
df

In [ ]:
data  = df.with_columns(
    iq=(pl.col("rule_of_law") + pl.col("control_of_corruption") + pl.col("regulatory_quality") + pl.col("political_stability") + pl.col("voice") + pl.col("government_effect")) / 6
)
data = data.filter(pl.col("advanced_economy") == 0)
IQ = data.select(pl.col("iq").median()).item()

data = data.group_by("country").agg(
    iq=pl.col("iq").mean()
)
data  = data.with_columns(
    iq_dummy = pl.when(pl.col("iq") >= IQ).then(1).otherwise(0)
    )
dataset = df.join(data, on="country", how="inner", validate="m:1")
dataset = df.with_columns(
    year_id=pl.col("year").rank(method="min").over("country"),
    county_id=pl.col("country").rank(method="min").over("year")
)
dataset

In [ ]:
# add year as continues id and add country id order by alphabetically 

In [ ]:
data.select(pl.col("iq_dummy").sum())

In [ ]:
temp = df.select(pl.col("year", "country"))
temp